In [ ]:
# 🚗 Car Parts Image Generator (DreamShaper + CSV)
# Google Colab notebook

# --- 1. Setup ---
!pip install diffusers transformers accelerate safetensors pandas

import os
import pandas as pd
from diffusers import AutoPipelineForText2Image, DPMSolverMultistepScheduler
import torch
from google.colab import drive

# --- 2. Mount Google Drive ---
# This step requires authentication. Follow the prompts when you run the cell.
drive.mount('/content/drive')

# --- 3. Configuration ---
# Path to your CSV file on Google Drive
csv_path = "/content/drive/MyDrive/car_parts/05_01_car_parts_game_style_100.csv"
# Folder on Google Drive to save the generated images
output_dir = "/content/drive/MyDrive/car_parts/generated_images_dreamshaper"
os.makedirs(output_dir, exist_ok=True)

# --- 4. Load CSV ---
df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} car parts.")

# --- 5. Load DreamShaper XL Lightning model ---
# Using AutoPipelineForText2Image for flexibility and compatibility.
# DreamShaper XL Lightning is a fast model, allowing for fewer inference steps.
model_id = "lykon/dreamshaper-xl-lightning"
pipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
# Using a specific scheduler recommended for this model
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

# --- 6. Generate images ---
# Parameters like num_inference_steps and guidance_scale are tuned for the DreamShaper model.
for idx, row in df.iterrows():
    part_name = row.get("Part") or row.get("Part Name") or f"part_{idx}"
    part_description = row.get("Description")
    # A more descriptive prompt tailored for better artistic results with DreamShaper.
    prompt = (
        f"masterpiece, best quality, photorealistic, game asset, futuristic car part, "
        f"{part_name}: {part_description}, "
        f"8k, detailed, intricate, elegant, highly detailed, sharp focus, vibrant colors"
    )

    try:
        # Generate the image with specified size and parameters
        image = pipe(
            prompt,
            num_inference_steps=6,  # Slightly more steps for potentially better quality
            guidance_scale=2.5,
            width=512,
            height=512
        ).images[0]
        filename = f"{part_name.replace(' ', '_')}.png"
        filepath = os.path.join(output_dir, filename)
        image.save(filepath)
        print(f"✅ Generated {part_name} → {filepath}")
    except Exception as e:
        print(f"❌ Failed for {part_name}: {e}")

print("🎉 Done! All images saved to Google Drive.")